In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time
from urllib.request import urlretrieve
from tqdm import tqdm
import os,csv
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [2]:
def driver_start(university):
    chrome_options = webdriver.ChromeOptions() 
    chrome_options.add_argument('--headless')#창없이 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    path=r'/data/chromedriver'
    driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
    driver.set_window_size(1024,1000)

    url = "https://www.yogiyo.co.kr/mobile/#/"
    driver.get(url)
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').clear()
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys(university)
    driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()
    time.sleep(1)
    
    return driver

In [3]:
def scroll_element(shop):#음식점이 있는 곳까지 스크롤
    print(f"{shop} scroll")
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        try:
            some_tag = driver.find_element_by_xpath(f'//*[@title="{shop}"]')
            if some_tag:
                return False
        except Exception:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: #끝까지 스크롤후 해당 음식점이 없으면 종료 
                return True
            last_height = new_height
            time.sleep(0.5)

In [4]:
def stretch_review_page(review_cnt):
    #클린리뷰에서 제일 아랫부분까지 열기
    click_count = int((review_cnt/10))
    print('모든 리뷰 불러오기 시작...')
    for _ in range(click_count):
        try:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            driver.find_element_by_class_name('btn-more').click()
            time.sleep(2)
        except Exception as e:
            pass
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    print('모든 리뷰 불러오기 완료!')

In [5]:
def craw(i,shop,driver,col_list,cata):
    #클린리뷰클릭
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]').click()
    except Exception:
        driver.quit()
        driver=driver_start(university)
        time.sleep(3)
        scroll_element(shop)
        #driver.find_element_by_xpath(f'//*[@title="{shop_in[i]}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]').click()
    
    time.sleep(2)
    review_cnt=driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[2]/a/span').text
    review_cnt=int(review_cnt)

    print(shop, review_cnt)
    
    if review_cnt==0: #리뷰없으면 종료
        print(0)
        return
    
    stretch_review_page(review_cnt)
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    b=soup.find_all('li',{'class':'list-group-item star-point ng-scope'})
    
    tot_star_list=[]
    name_list,date_list,star_flavor_list,star_volume_list,star_deliver_list=[],[],[],[],[]
    menu_list,review_customer_list,review_owner_list,img_name_list=[],[],[],[]
    for i in tqdm(range(len(b)),desc='save..',mininterval=3):
        now=b[i]
        name=now.find('span',{'ng-show':"review.phone"}).text
        name_list.append(name)
        date=now.find('span',{'ng-bind':"review.time|since"}).text
        date_list.append(date)
        
        try:
            star=now.find_all('span',{'class':"points ng-binding"})
        except IndexError:
            print(f'{i}th review, none star')
            continue
        
        
        try:
            tot_star=len(now.find_all('span',{'class':"full ng-scope"}))
        except IndexError:
            print(f'{i}th review, none star')
            continue
        tot_star_list.append(tot_star)
        
        try:
            star_flavor=star[0].text
        except IndexError:
            star_flavor=None
        star_flavor_list.append(star_flavor)
        
        try:
            star_volume=star[1].text
        except IndexError:
            star_volume=None
        star_volume_list.append(star_volume)

        try:
            star_deliver=star[2].text
        except IndexError:
            star_deliver=None
            pass
        star_deliver_list.append(star_deliver)
            
        menu=now.find("div",{"class":"order-items default ng-binding"}).text.strip()
        menu_list.append(menu)
        review_customer=now.find("p",{"class":"ng-binding"}).text
        review_customer_list.append(review_customer)

    
        try:
            review_owner=now.find("p",{"ng-bind-html":"review.owner_reply.comment|strip_html"}).text
        except AttributeError:
            review_owner=None
        review_owner_list.append(review_owner)
        try:
            img_urls=now.find_all('img')
        except TypeError:
            img_names=None
        else:
            img_names=[]
            for idx in range(len(img_urls)):
                img_name=shop+'_'+str(i)+'_'+str(idx)
                path=f'/data/yogiyo/pr_data/uos/cata{cata}/{shop}'
                img_url=img_urls[idx]['src']
                if not os.path.isdir(path):
                    os.mkdir(path)
                try:
                    urlretrieve(img_url,path+'/'+img_name+'.jpg')
                except requests.exceptions.SSLError or URLError:
                    pass
                img_names.append(img_name)
        img_name_list.append(img_names)
                
    #new=pd.DataFrame([[shop,name,date,star_flavor,star_volume,star_deliver,menu,review_customer,review_owner,img_name]],columns=col_list)
    data={'shop':shop,'name':name_list,'date':date_list,'tot_star':tot_star_list,
          'star_flavor':star_flavor_list,'star_volume':star_volume_list,
         'star_deliver':star_deliver_list,'menu':menu_list,'review_customer':review_customer_list,
          'review_owner':review_owner_list,'img_name':img_name_list}
    new=pd.DataFrame(data)

    return new

In [6]:
path='/data/yogiyo/pr_data/uos/'
university='서울특별시 동대문구 전농동 694 서울시립대학교'
uni_shop=pd.read_csv(path+'uos_shop.csv')

In [7]:
uni_shop.catagory.value_counts()

4    882
0    276
3     94
1     70
2     66
Name: catagory, dtype: int64

In [8]:
cata=4
start=300
end=600


shops_list=uni_shop.shop[uni_shop.catagory == cata].tolist()
col_list=['shop','name','date','tot_star','star_flavor','star_volume','star_deliver','menu','review_customer','review_owner','img_name']
df=pd.DataFrame(columns=col_list)
global driver
#driver.quit()
driver=driver_start(university)
for i in tqdm(range(start,end),desc='craw'):
    shop=shops_list[i]
    go_stop=scroll_element(shop)
    if go_stop:
        #스크롤 했는데 음식점 없으면 다음으로 넘어감
        print('wrong shop')
        continue
    
    time.sleep(3)
    
        #찾은 음식점 클릭
    try:
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
    except (Exception, NoSuchElementException, ElementClickInterceptedException):
        driver.quit()
        driver=driver_start(university)
        time.sleep(2)
        scroll_element(shop)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop}"]').click()
    time.sleep(3)
    

    new=craw(i,shop,driver,col_list,cata)
    df=pd.concat([new,df])
    print(df.shape)
    driver.back()
    time.sleep(2)

print(f'tot_shop:{i},crawling sucess')

<ipython-input-2-bb62db29bb57>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
craw:   0%|          | 0/300 [00:00<?, ?it/s]

배스킨라빈스-경희대점 scroll
배스킨라빈스-경희대점 123
모든 리뷰 불러오기 시작...
모든 리뷰 불러오기 완료!



save..: 100%|██████████| 123/123 [00:06<00:00, 18.87it/s][A


(123, 11)


craw:   0%|          | 1/300 [01:23<6:56:07, 83.50s/it]

...

롯데리아-장안사거리점 scroll
롯데리아-장안사거리점 1479
모든 리뷰 불러오기 시작...
모든 리뷰 불러오기 완료!



save..: 100%|██████████| 1479/1479 [00:41<00:00, 36.00it/s]


(52734, 11)


craw: 100%|██████████| 300/300 [6:45:19<00:00, 81.06s/it] 

tot_shop:599,crawling sucess


In [9]:
df.to_csv(path+f'cata{cata}_2.csv',index=False)